In [1]:
import os

from langchain_community.graphs import Neo4jGraph

os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "2wsx3edc"
database = os.environ.get('NEO4J_DATABASE')
graph = Neo4jGraph(database=database)

In [5]:
from langchain_openai import AzureOpenAIEmbeddings

embedding = AzureOpenAIEmbeddings(
    model="text-embedding-3-small",
    azure_endpoint='https://sales-chatbot-llm.openai.azure.com/openai/deployments/text-embedding-3-small/embeddings?api-version=2023-05-15',
    azure_deployment='text-embedding-3-small',
    openai_api_version='2023-05-15'
)

In [35]:
# from langchain_community.vectorstores import Neo4jVector
# # ! pip3 install -U langchain-huggingface
# import os
# os.environ['SENTENCE_TRANSFORMERS_HOME'] = '/storage/models/embedding_models'
# from langchain_huggingface import HuggingFaceEmbeddings
# # Choose from https://huggingface.co/spaces/mteb/leaderboard

# # embedding = HuggingFaceEmbeddings(model_name="lier007/xiaobu-embedding-v2")

# model_path = os.path.join(os.environ['SENTENCE_TRANSFORMERS_HOME'], 'models--lier007--xiaobu-embedding-v2/snapshots/ee0b4ecdf5eb449e8240f2e3de2e10eeae877691')
# embedding = HuggingFaceEmbeddings(model_name=model_path)

In [36]:
# node_label='__Entity__'
# embedding_node_property='embedding'
# fetch_query = (
#     f"MATCH (n:`{node_label}`) "
#     f"WHERE n.{embedding_node_property} IS null "
#     "AND any(k in $props WHERE n[k] IS NOT null) "
#     f"RETURN elementId(n) AS id, reduce(str='',"
#     "k IN $props | str + '\\n' + k + ':' + coalesce(n[k], '')) AS text "
#     "LIMIT 1000"
# )
# datas = graph.query(fetch_query, params={"props": ['id', 'description']})
# datas

In [37]:
# import sys
# sys.path.append('..')
# from tools.TokenCounter import num_tokens_from_string

# tokens_num = 0
# for data in datas:
#     tokens_num += num_tokens_from_string(data['text'])
# tokens_num

In [ ]:
from langchain_community.vectorstores import Neo4jVector

lc_retrieval_query = """
WITH collect(node) as nodes
// Entity - Text Unit Mapping
WITH
collect {
    UNWIND nodes as n
    MATCH (n)<-[:HAS_ENTITY]->(c:__Chunk__)<-[:HAS_CHILD]->(p:__Parent__)
    WITH c, p, count(distinct n) as freq
    RETURN {content: p.content, source: p.source} AS chunkText
    ORDER BY freq DESC
    LIMIT $topChunks
} AS text_mapping,
// Entity - Report Mapping
collect {
    UNWIND nodes as n
    MATCH (n)-[:IN_COMMUNITY]->(c:__Community__)
    WHERE c.summary is not null
    WITH c, c.rank as rank, c.weight AS weight
    RETURN c.summary 
    ORDER BY rank, weight DESC
    LIMIT $topCommunities
} AS report_mapping,
// Outside Relationships 
collect {
    UNWIND nodes as n
    MATCH (n)-[r]-(m) 
    WHERE NOT m IN nodes and r.description is not null
    RETURN {description: r.description, sources: r.sources} AS descriptionText
    ORDER BY r.rank, r.weight DESC 
    LIMIT $topOutsideRels
} as outsideRels,
// Inside Relationships 
collect {
    UNWIND nodes as n
    MATCH (n)-[r]-(m) 
    WHERE m IN nodes and r.description is not null
    RETURN {description: r.description, sources: r.sources} AS descriptionText
    ORDER BY r.rank, r.weight DESC 
    LIMIT $topInsideRels
} as insideRels,
// Entities description
collect {
    UNWIND nodes as n
    match (n)
    WHERE n.description is not null
    RETURN {description: n.description, sources: n.sources} AS descriptionText
} as entities
// We don't have covariates or claims here
RETURN {Chunks: text_mapping, Reports: report_mapping, 
       Relationships: outsideRels + insideRels, 
       Entities: entities} AS text, 1.0 AS score, {} AS metadata
"""

vectorstore = Neo4jVector.from_existing_graph(embedding=embedding, 
                                    index_name="embedding",
                                    node_label='__Entity__', 
                                    embedding_node_property='embedding', 
                                    text_node_properties=['id', 'description'],
                                    retrieval_query=lc_retrieval_query)

In [24]:
topChunks = 3
topCommunities = 3
topOutsideRels = 10
topInsideRels = 10
topEntities = 10

In [25]:
os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "2wsx3edc"

In [26]:
docs = vectorstore.similarity_search(
    "繳費方式有哪些?",
    k=topEntities,
    params={
        "topChunks": topChunks,
        "topCommunities": topCommunities,
        "topOutsideRels": topOutsideRels,
        "topInsideRels": topInsideRels,
    },
)
print(docs[0].page_content)

Entities:
- 操作方式詳官網
Reports:
- 繳費方式與相關文件之間存在檢附關係。來源檔名: 個人保險首續期繳費暨保單貸款還款息作業手冊(113年7月版)_業務通路版_p3_p7.pdf
- 繳款人相關規定包含繳款人身分、自動轉帳及信用卡付款授權書作業規範【加強身分驗證機制】以及繳費方式及受理應檢附文件。這些資訊均來自《個人保險首續期繳費暨保單貸款還款息作業手冊(113年7月版)_業務通路版_p3_p7.pdf》。
- 繳款人相關規定包含繳款人身分、自動轉帳及信用卡付款授權書作業規範【加強身分驗證機制】以及繳費方式及受理應檢附文件。這些資訊均來自《個人保險首續期繳費暨保單貸款還款息作業手冊(113年7月版)_業務通路版_p3_p7.pdf》。
Chunks:
- {'content': '回目錄\n目錄\n壹、 繳交保險費應注意事項 . 1\n貳、 繳款人相關規定 . 3\n一、 繳款人身分 .. 3\n二、 繳費方式及受理應檢附文件  3\n三、 自動轉帳及信用卡付款授權書作業規範【加強身分驗證機制】  4\n參、 首期保險費繳交方式及相關規定  6\n一、 首期保險費-採金融機構轉帳作業  6\n二、 首期保險費-採信用卡扣款作業  7\n三、 首期保險費-採自行匯款作業 . 8\n四、 首期保險費-採自行匯款管道 . 8\n(一) 台幣-首期匯款帳號【業務通路適用】  8\n(二) 外幣-首期匯款帳號【業務通路適用】  9\n(三) WEB立即繳費 . 10\n(四) 中信ATM繳費(操作方式詳官網) .. 10\n肆、 續期保險費繳交方式及相關規定 . 11\n一、 續期保險費-採金融機構轉帳作業 .. 11\n二、 續期保險費-採信用卡扣款作業 .. 13\n三、 續期保險費-採自行繳費作業  15\n四、 高齡關懷電訪作業 .. 17\n五、 續期保險費扣款不成通知方式 . 18\n伍、 各金融機構核印、扣款時間及相關規定 . 21\n陸、 保單貸款還款息 .. 23\n一、 保單貸款之繳息通知 . 23\n二、 保單借款之還款息帳號  24\n柒、 行動保全續期保費服務說明(僅適用業務通路)  25\n版次：113年07月版 修訂日期：113.07.01\n回目錄\n壹、 繳交保險費應注意事項\n一、 因應「保險業保險經紀人公司及保險代

In [42]:
docs

[Document(metadata={}, page_content='Entities:\n- 匯款帳號\n- 業務通路適用\n- 匯款帳號\n- 匯款帳號\n- 匯款帳號\n- 匯款帳號\n- 匯款帳號\n- 首期保險費採自行匯款者，可透過銀行/郵局以匯款/劃撥方式繳交，並可利用 e-bill首期立即繳費及中信ATM 繳費。\n- 匯款銀行\nReports:\n- 在同一社區中，有兩個概念節點。第一個節點是「首期保險費-採自行匯款管道」，第二個節點是「外幣-首期匯款帳號」。這兩個節點之間存在一個關係，即「首期保險費-採自行匯款管道」包含「外幣-首期匯款帳號」。\n- 在這個社區中，有兩個主要的節點。第一個節點是 "Ctcbtwtpxxx"，它是一個識別碼，具體描述為 SWIFT CODE。第二個節點是 "Ctbc Bank Co.,Ltd."，它是一個組織，具體描述為匯款銀行。這兩個節點之間存在一個關係，即 "Ctbc Bank Co.,Ltd." 擁有 "Ctcbtwtpxxx" 這個 SWIFT CODE。\n- 台幣首期保險費是一個概念，其描述為依轄屬分公司匯入以下指定台幣實體帳號。台幣-首期匯款帳號也是一個概念，其描述為業務通路適用。這兩個概念之間存在一個關係，即台幣首期保險費會轉帳至台幣-首期匯款帳號。\nChunks:\n- 4. **支票**\n   - 業務人員無代收保戶支票之授權。\n目錄\n壹、 繳交保險費應注意事項 . 1\n貳、 繳款人相關規定 . 3\n一、 繳款人身分 .. 3\n二、 繳費方式及受理應檢附文件  3\n三、 自動轉帳及信用卡付款授權書作業規範【加強身分驗證機制】  4\n參、 首期保險費繳交方式及相關規定  6\n一、 首期保險費-採金融機構轉帳作業  6\n二、 首期保險費-採信用卡扣款作業  7\n三、 首期保險費-採自行匯款作業 . 8\n四、 首期保險費-採自行匯款管道 . 8\n(一) 台幣-首期匯款帳號【業務通路適用】  8\n(二) 外幣-首期匯款帳號【業務通路適用】  9\n(三) WEB立即繳費 . 10\n(四) 中信ATM繳費(操作方式詳官網) .. 10\n肆、 續期保險費繳交方式及相關規定 . 11\n一、 續期保險費-採金融機構轉帳作業 .. 11\n二、 續期保險費-採信用卡扣款作業